# Update photo saving

In [1]:

import mimetypes

In [2]:
mimetypes.guess_type('game.gif')

('image/gif', None)

In [3]:
from config import Config

def filtration_by_filename(photo_filename: str) -> bool:
    for rule in Config.filtration_keywords:
        if all(keyword in photo_filename for keyword in rule):
                return True
    return False

print(filtration_by_filename('img_0_1_coursead-dlsoptimizationalgorithms'))
print(filtration_by_filename('issue_1_title_bert_is_back_img_0_1_cropped20roberta'))


True
False


In [6]:
import os
import json
import requests
from urllib.parse import urlparse
import re
from config import Config
from tqdm import tqdm

with open(f'../{Config.parsed_full_data_path}', 'r', encoding='utf-8') as f:
    articles = json.load(f)

output_dir = f'../{Config.image_dataset_path}'
os.makedirs(output_dir, exist_ok=True)


def slugify(text):
    if text is not None:
        text = text.lower().strip().replace(' ', '_')
    else:
        text = 'no_text'
    return re.sub(r'[^a-zA-Z0-9_\-]', '', text)


def filtration_by_filename(photo_filename: str) -> bool:
    for rule in Config.filtration_keywords:
        if all(keyword in photo_filename for keyword in rule):
                return True
    return False

uris = []
metadata = [] # now is only url and title

# Download images
for article in tqdm(articles):
    url = article.get('url', 'None')
    issue = article.get('issue', 'unknown')
    title_slug = slugify(article.get('title', 'no_title'))

    for idx, img_url in enumerate(article.get('images', [])):
        try:
            parsed_url = urlparse(img_url)
            img_name = os.path.basename(parsed_url.path)
            img_ext = os.path.splitext(img_name)[-1] or '.jpg'
            img_file_slug = slugify(os.path.splitext(img_name)[0])
            filename = f'issue_{issue}_title_{title_slug}_img_{idx}_{img_file_slug}{img_ext}'
            if filtration_by_filename(filename):
                print(f"Filtered: {filename}")
                continue
  
            filepath = os.path.join(output_dir, filename)
            
            # load and save image from url
            response = requests.get(img_url, timeout=10)
            response.raise_for_status()
            with open(filepath, 'wb') as f:
                f.write(response.content)
            print(f"Saved: {filename}")
            
            # save data parameters into json
            uris.append(filename)
            print({'url': url, 'issue':issue, 'title':title_slug})
            metadata.append({'url': url, 'issue':issue, 'title':title_slug})
        except Exception as e:
            print(f"Error downloading {img_url}: {e}")

export_data = {
    "uris": uris,
    "metadata": metadata
}

with open(f'../{Config.dataset_path}/photo_data.json', 'w') as f:
    json.dump(export_data, f)

print(f"Data exported to {Config.dataset_path}/photo_data.json")

100%|██████████| 1134/1134 [00:00<00:00, 35394.71it/s]

issue_1_title_this_shirt_hates_surveillance_img_0_adversarial20fashion.gif
{'url': 'https://www.deeplearning.ai/the-batch/issue-1/', 'issue': 1, 'title': 'this_shirt_hates_surveillance'}
issue_1_title_deepmind_results_raise_questions_img_0_1_deepmind.jpg
{'url': 'https://www.deeplearning.ai/the-batch/issue-1/', 'issue': 1, 'title': 'deepmind_results_raise_questions'}
issue_1_title_bert_is_back_img_0_1_cropped20roberta.png
{'url': 'https://www.deeplearning.ai/the-batch/issue-1/', 'issue': 1, 'title': 'bert_is_back'}
issue_1_title_a_message_fromdeeplearningai_img_0_1_coursead-dlsoptimizationalgorithms.png
Filtered: issue_1_title_a_message_fromdeeplearningai_img_0_1_coursead-dlsoptimizationalgorithms.png
issue_1_title_standards_in_the_making_img_0_1_nist.png
{'url': 'https://www.deeplearning.ai/the-batch/issue-1/', 'issue': 1, 'title': 'standards_in_the_making'}
issue_1_title_style_upgrade_img_0_1_style20320sized-1024x577.png
{'url': 'https://www.deeplearning.ai/the-batch/issue-1/', 'issu

919

In [8]:
import os
import json
import requests
from urllib.parse import urlparse
import re
from config import Config
from tqdm import tqdm
from PIL import Image
from io import BytesIO

# Load articles
with open(f'../{Config.parsed_full_data_path}', 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Prepare output directory
output_dir = f'../{Config.image_dataset_path}'
os.makedirs(output_dir, exist_ok=True)

def slugify(text):
    if text is not None:
        text = text.lower().strip().replace(' ', '_')
    else:
        text = 'no_text'
    return re.sub(r'[^a-zA-Z0-9_\-]', '', text)

def filtration_by_filename(photo_filename: str) -> bool:
    for rule in Config.filtration_keywords:
        if all(keyword in photo_filename for keyword in rule):
            return True
    return False

# Download images
uris = []
metadata = []

for article in tqdm(articles):
    url = article.get('url', 'None')
    issue = article.get('issue', 'unknown')
    title_slug = slugify(article.get('title', 'no_title'))

    for idx, img_url in enumerate(article.get('images', [])):
        try:
            parsed_url = urlparse(img_url)
            img_name = os.path.basename(parsed_url.path)
            img_ext = os.path.splitext(img_name)[-1] or '.jpg'
            img_file_slug = slugify(os.path.splitext(img_name)[0])
            filename = f'issue_{issue}_title_{title_slug}_img_{idx}_{img_file_slug}{img_ext}'

            if filtration_by_filename(filename):
                #print('filtrated:', filename)
                continue

            filepath = os.path.join(output_dir, filename)

            # Step 1: Download
            response = requests.get(img_url, timeout=10)
            response.raise_for_status()

            # Step 2: Check Content-Type is image
            if 'image' not in response.headers.get('Content-Type', ''):
                continue

            # Step 3: Validate the image can be opened
            try:
                img = Image.open(BytesIO(response.content))
                img.verify()  # Checks for corrupted image
            except Exception:
                continue  # Skip invalid images

            # Step 4: Save the validated image
            with open(filepath, 'wb') as f:
                f.write(response.content)

            # Save metadata
            uris.append(filename)
            metadata.append({'url': url, 'issue': issue, 'title': title_slug})

        except Exception:
            # Skip image if any error occurs
            continue

# Save metadata to JSON
export_data = {
    "uris": uris,
    "metadata": metadata
}

with open(f'../{Config.dataset_path}/photo_data.json', 'w', encoding='utf-8') as f:
    json.dump(export_data, f, ensure_ascii=False, indent=4)


  0%|          | 3/1134 [00:02<16:06,  1.17it/s]

filtrated: issue_1_title_a_message_fromdeeplearningai_img_0_1_coursead-dlsoptimizationalgorithms.png


  2%|▏         | 22/1134 [00:17<12:32,  1.48it/s]

filtrated: issue_5_title_a_message_fromdeeplearningai_img_0_1_dls20course20420course20ad20fixed20size.png


  3%|▎         | 36/1134 [00:27<14:13,  1.29it/s]

...

 99%|█████████▉| 1126/1134 [07:07<00:01,  7.12it/s]

filtrated: issue_238_title_blazing_inference_speed_img_0_groq-llmperf.webp
filtrated: issue_238_title_a_message_fromdeeplearningai_img_0_the-batch-ads-and-exclusive-banners---2024-02-20t090733220.png


100%|█████████▉| 1132/1134 [07:08<00:00,  6.76it/s]

filtrated: issue_239_title_a_message_fromdeeplearningai_img_0_the-batch-ads-and-exclusive-banners---2024-03-05t085620866.png


100%|██████████| 1134/1134 [07:08<00:00,  2.64it/s]


# Web scraping

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import json

from config import Config

def setup_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument('--headless=new')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    driver = webdriver.Chrome(options=options)
    return driver

def extract_issue_data(driver, issue_number):
    url = f"https://www.deeplearning.ai/the-batch/issue-{issue_number}/"
    print(url)
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_content = soup.find("div", class_="prose--styled")

    if not main_content:
        print(f"❌ No main content found for issue {issue_number}")
        return None

    news_anchor = main_content.find(lambda tag: tag.name in ["h1", "h2"] and tag.get("id") == "news")

    if not news_anchor:
        print(f"⚠️ No 'news' anchor found in issue {issue_number}, get all")
        elements_to_process = main_content
    else:
        elements_to_process = list(news_anchor.next_siblings)

    articles_data = []
    current_article = {
        "issue": issue_number,
        "url": url,
        "title": None,
        "text": "",
        "images": [],
        "captions": []
    }

    for element in elements_to_process:
        if isinstance(element, str):
            continue

        if element.name in ["h1", "h2", "h3"]:
            current_article["title"] = element.get_text(strip=True)

        elif element.name == "p":
            text = element.get_text(strip=True)
            if text:
                current_article["text"] += text + "\n"

        elif element.name == "ul":
            for li in element.find_all("li"):
                li_text = li.get_text(strip=True)
                if li_text:
                    current_article["text"] += "• " + li_text + "\n"

        elif element.name == "figure":
            img_tag = element.find("img")
            if img_tag:
                img_src = img_tag.get("src")
                if img_src:
                    current_article["images"].append(img_src)

            caption_tag = element.find("figcaption")
            if caption_tag:
                caption = caption_tag.get_text(strip=True)
            else:
                # fallback — сусідній <p>
                next_p = element.find_next_sibling("p")
                caption = next_p.get_text(strip=True) if next_p else ""

            current_article["captions"].append(caption)

        elif element.name == "hr":
            if current_article["title"] is not None or current_article["text"] is not None:
                if 'A MESSAGE FROM' not in current_article["title"]:
                    articles_data.append(current_article)
                current_article = {
                    "issue": issue_number,
                    "url": url,
                    "title": None,
                    "text": "",
                    "images": [],
                    "captions": []
                }

    if current_article["title"] or current_article["text"]:
        articles_data.append(current_article)

    return articles_data

def scrape_all_issues(start=1, end=239, headless=True):
    driver = setup_driver(headless)
    all_issues = []

    for issue_number in range(start, end + 1, 1):
        try:
            print(f"Scraping Issue #{issue_number}")
            data = extract_issue_data(driver, issue_number)
            all_issues.extend(data)
        except Exception as e:
            print(f"Failed to scrape issue {issue_number}: {e}")
            continue

    driver.quit()
    return all_issues


results = scrape_all_issues(start=39, end=39)

print("✅ Scraping complete. Data saved to 'the_batch_articles.json'")
results

Scraping Issue #39
https://www.deeplearning.ai/the-batch/issue-39/
✅ Scraping complete. Data saved to 'the_batch_articles.json'


[{'issue': 39,
  'url': 'https://www.deeplearning.ai/the-batch/issue-39/',
  'title': 'Mask Monitor',
  'text': 'Cameras that detect face masks are helping French authorities to evaluate citizens’ adherence to government mandates intended to fight Covid-19.What’s new:Starting this week, everyone riding public transportation in France isrequiredto wear a face mask.Paris and Cannesare using computer vision to count people who comply.How it works:Datakalab, a French AI startup, is installing chips in existing CCTV cameras that run an object recognition model. The model is trained to distinguish masked faces from unmasked ones.\n• Paris istestingthe cameras at the busy Chatelet-Les Halles metro station. Cannes has installed them on buses and in public markets.\n• The software counts mask wearers every 15 minutes and transmits aggregate statistics to the authorities. The companysaysthe system is meant to help authorities determine where to step up efforts to promote mask-wearing\n• Datakala